# Exercice 8
## Utiliser des APIs
### Consigne
* Récupérer l'item policy d'un exemplaire 
* Modifier l'item policy de l'exemplaire vers 21 (2 semaines)

**Exercice avancé:** 
* Récupérer le user group pour un ensemble d'utilisateurs

In [39]:
# Importer les bibliothèques requises
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import json

PATH_TO_SECRETS = '../Secrets/'

In [40]:
# Chargement des données
df = pd.read_csv('Data/barcode_hph_sb.csv')

In [41]:
df.head()

,Barcode
0,HR40042121
1,02085863
2,03130885
3,02076508
4,1034139759


### Récupérer l'item policy
Documentation Alma:
* Récupérer les données d'un exemplaire: https://developers.exlibrisgroup.com/alma/apis/docs/bibs/R0VUIC9hbG1hd3MvdjEvYmlicy97bW1zX2lkfS9ob2xkaW5ncy97aG9sZGluZ19pZH0vaXRlbXMve2l0ZW1fcGlkfQ==/
* Modifier un exemplaire: https://developers.exlibrisgroup.com/alma/apis/docs/bibs/UFVUIC9hbG1hd3MvdjEvYmlicy97bW1zX2lkfS9ob2xkaW5ncy97aG9sZGluZ19pZH0vaXRlbXMve2l0ZW1fcGlkfQ==/
* Récupérer les données d'une holding: https://developers.exlibrisgroup.com/alma/apis/docs/bibs/R0VUIC9hbG1hd3MvdjEvYmlicy97bW1zX2lkfS9ob2xkaW5ncy97aG9sZGluZ19pZH0=/
* Modifier une holding: https://developers.exlibrisgroup.com/alma/apis/docs/bibs/UFVUIC9hbG1hd3MvdjEvYmlicy97bW1zX2lkfS9ob2xkaW5ncy97aG9sZGluZ19pZH0=/

In [48]:
# Clé Sandbox en lecture et écriture pour les ressources bibliographiques
with open(PATH_TO_SECRETS + 'API_KEY_ITEMS_SB') as f:
    # Il faut utiliser "strip" pour supprimer le saut de ligne à la fin
    API_KEY_BIB_SB = f.read().strip()

# URL de base de l'api des utilisateurs dans Alma
API_ITEM_URL = 'https://api-eu.hosted.exlibrisgroup.com/almaws/v1/items'
# En-têtes de la futur requête
# Définit notamment le format de la réponse
HEADERS = {'content-type': 'application/json',
           'accept': 'application/json',
           'Authorization': 'apikey ' + API_KEY_BIB_SB}

In [49]:
# Récupération de la cote en fonction du code-barre
barcode = 'HR40042121'
r1 = requests.get(API_ITEM_URL, params={'item_barcode': barcode}, headers=HEADERS)

data1 = r1.json()
data1['item_data']['policy']

{'value': '21', 'desc': '21 Loan 14 days'}

### Modifier l'item policy
Note: le plus simple est de remplacer le dictionnaire de l'item policy par un nouveau qui ne comporte que
"value". Le champ "desc" est facultatif pour effectuer une modification.

In [50]:
data1['item_data']['policy'] = {'value': '21'}
url = data1['link']

In [51]:
r2 = requests.put(url, headers=HEADERS, data=json.dumps(data1))
data2 = r2.json()
data2['item_data']['policy']

{'value': '01', 'desc': '01 Loan 28 days'}

## Exercice avancé
Le but de cet exercice est d'interroger l'API swisscovery pour récupérer tous les user groups
des participants du cours.

Quand il ne sont pas inscris, afficher "NO ACCOUNT".

La liste des participants se trouve dans les "Secrets".

LA documentation de l'API se trouve ici: https://developers.exlibrisgroup.com/alma/apis/users/

Pour réussir cette opération, il faut effectuer deux requêtes:
1. Rechercher l'utilisateur par email: https://developers.exlibrisgroup.com/alma/apis/docs/users/R0VUIC9hbG1hd3MvdjEvdXNlcnM=/
2. Recupérer le primary_id et rechercher les détails de l'utilisateur: https://developers.exlibrisgroup.com/alma/apis/docs/users/R0VUIC9hbG1hd3MvdjEvdXNlcnMve3VzZXJfaWR9/
3. Récupérer le user_group

### Solution simple avec "apply"
Créer une fonction qui récupère le user_group à partir de l'email
puis créer une nouvelle colonne en utilisant "apply"

In [6]:
# Créer un DataFrame en récupérant les la liste des participants dans les secrets
df = pd.read_csv(PATH_TO_SECRETS + 'participants.csv')

In [7]:
# Clé NZ en lecture pour les utilisateurs
with open(PATH_TO_SECRETS + 'API_KEY_USERS_NZ') as f:
    API_KEY_USERS_NZ = f.read().strip()

# URL de base de l'api des utilisateurs dans Alma
API_BASE_URL = 'https://api-eu.hosted.exlibrisgroup.com/almaws/v1/users/'

# En-têtes de la futur requête
# Définit notamment le format de la réponse
HEADERS = {'content-type': 'application/json',
           'accept': 'application/json',
           'Authorization': 'apikey ' + API_KEY_USERS_NZ}

def get_user_group_from_email(email):
    # Il faut interroger l'API en faisant une requête avec le paramètre "q"
    r1 = requests.get(API_BASE_URL, params={'q': f'email~{email}'}, headers=HEADERS)
    
    # r1.ok vaut True si la requête a retourné un code de réponse inférieur à 400
    if r1.ok is not True:
        return 'ERROR'
    
    data = r1.json()
    
    # S'il n'y a pas d'utilisateur, il n'y aura pas de clé "user".
    # Une exception "KeyError" sera générée et interceptée
    try:
        link = data['user'][0]['link']
    except KeyError:
        return 'NO ACCOUNT'
    
    r2 = requests.get(link, headers=HEADERS)
    if r2.ok is not True:
        return 'ERROR'
    
    try:
        user_group = r2.json()['user_group']['desc'].strip()
        
    except KeyError:
        user_group = ''
    
    return user_group

# Crée une colonne à l'aide d'apply
df['User group'] = df['Email'].apply(get_user_group_from_email)

**Remarque importante**
Ne jamais faire ainsi sur de grands sets de données.

Si le programme plante, tout le traitement serait perdu.

Si le set n'est pas trop grand, on peut itérer avec iterrows et
enregistrer les résultats au fur et à mesure dans un nouveau
DataFrame.

### Solution plus complexe avec "iterrows"
Le but cette fois-ci est d'enregistrer un nouveau DataFrame après le traitement
de chaque ligne. "iterrows" permet de parcourir les lignes d'un DataFrame, mais il ne faut
pas le modifer lorsqu'on le parcourt.

Il faut donc créer un nouveau DataFrame et l'enregistrer à chaque itération.

Si le programme plante, on pourra ainsi récupérer le travail déjà effectué.

In [10]:
df = pd.read_csv(PATH_TO_SECRETS + 'participants.csv')
d_temp = pd.DataFrame(columns=['Email', 'User group'])

for row in df.iterrows():
    # iterrows retourne des "Tuple" de type (<numéro_ligne>, {<colonne_B>: <valeur>})
    email = row[1]['Email']
    user_group = get_user_group_from_email(email)
    d_temp.loc[len(d_temp)] = [email, user_group]
    d_temp.to_csv('Resultat/participants_ave_user_group.csv')

In [13]:
# Ici cela ne fait pas de sens mais ensuite l'idée serait d'utiliser "merge" pour fusionner
# les deux tables sur la base d'un identifiant unique
d_temp = d_temp.loc[:3]
d_temp

,Email,User group
0,myriam.benichou@unige.ch,"UNIGE-Staff, Student"
1,raphael.grolimund@hesge.ch,Academic Staff
2,romain.guedj@fr.ch,NO ACCOUNT
3,jules.guemara@ville-ge.ch,Swiss Resident


In [14]:
# Variante pour reprendre un processus en cours
df = pd.read_csv(PATH_TO_SECRETS + 'participants.csv')

# Au moment de l'enregistrement en csv, on a laissé l'index. Il faut donc mettre
# la première colonne en tant qu'index.
d_temp = pd.read_csv('Resultat/participants_ave_user_group.csv', index_col=0)
d_temp = d_temp.loc[:3]

for row in df.iterrows():
    
    # "continue" permet de passer directement à l'itération suivante
    if row[0] < len(d_temp):
        print(f'{row[0] + 1} / {len(df)}: "{row[1]["Email"]}" passé')
        continue
    
    # Essentiel de faire des logs
    print(f'{row[0] + 1} / {len(df)}: traitement "{row[1]["Email"]}"')
    
    email = row[1]['Email']
    user_group = get_user_group_from_email(email)
    
    # Motif très important pour ajouter une nouvelle ligne
    # à un DataFrame. Cela fonctionne seulement si on a
    # laissé l'index avec le numéro des lignes (comme
    # c'est le cas par défaut). Dans le cas contraire, il
    # faut mettre à la place du rang une valeur unique
    # d'index
    d_temp.loc[len(d_temp)] = [email, user_group]
    
    d_temp.to_csv('Resultat/participants_ave_user_group.csv')

1 / 10: "myriam.benichou@unige.ch" passé
2 / 10: "raphael.grolimund@hesge.ch" passé
3 / 10: "romain.guedj@fr.ch" passé
4 / 10: "jules.guemara@ville-ge.ch" passé
5 / 10: traitement "jasmin.huegi@unige.ch"
6 / 10: traitement "johan.mottet@gmail.com"
7 / 10: traitement "gregoire.savary@hefr.ch"
8 / 10: traitement "gabrielle.segev@unige.ch"
9 / 10: traitement "jeremie.theodoloz@ville-ge.ch"
10 / 10: traitement "maaike.vautier@bcu.unil.ch"


In [27]:
d_temp

,Email,User group
0,myriam.benichou@unige.ch,"UNIGE-Staff, Student"
1,raphael.grolimund@hesge.ch,Academic Staff
2,romain.guedj@fr.ch,NO ACCOUNT
3,jules.guemara@ville-ge.ch,Swiss Resident
4,jasmin.huegi@unige.ch,"UNIGE-Staff, Student"
5,johan.mottet@gmail.com,NO ACCOUNT
6,gregoire.savary@hefr.ch,Academic Staff
7,gabrielle.segev@unige.ch,"UNIGE-Staff, Student"
8,jeremie.theodoloz@ville-ge.ch,NO ACCOUNT
9,maaike.vautier@bcu.unil.ch,Swiss Resident
